### Program to calculate all things diamonds

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import math
cwd = os.getcwd()

# this gives you transmission and heat load for bpms at whatever conditions you tell it. 

##############################################
#Get elemental data in and properly formed
##############################################
carbon = np.loadtxt((cwd + "/sourceData/carbon.txt"), dtype = float,comments ="#",)
pt = np.loadtxt((cwd + "/sourceData/pt.txt"), dtype = float,comments ="#",)
al = np.loadtxt((cwd + "/sourceData/al.txt"), dtype = float,comments ="#",)
carbonT = np.loadtxt((cwd + "/sourceData/carbonTrans.txt"), dtype = float,comments ="#",)
ptT = np.loadtxt((cwd + "/sourceData/PtTrans.txt"), dtype = float,comments ="#",)
alT = np.loadtxt((cwd + "/sourceData/alTrans.txt"), dtype = float,comments ="#",)
###############################################
# Constants
###############################################
densC = 3.15 #g/cm^3
densPt = 21.4 #g/cm^3
densAl = 2.7 #g/cm^3
enConv = 1.60218e-19 #eV per Joule
coulPerElec =  6.28e18  #elec per s for amp
micro_cm = 1e-4 # microns to cm conversion
nano_cm = 1e-7 # nm to cm conversion
ToMev= 1e-6
freeCarrier = 13.2 #energy to make electorn hole pairs in diamond
################################################

#### Define functions for Diamond Calculations

In [2]:
def MAdevice_perf_al (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    coefAl = np.interp((energy*ToMev), al[:,0], al[:,2])
    transAl = (math.exp(-coefAl*metal_thick*nano_cm*densAl))
    alLayerAbs = (flux * (1-transAl))
    alLayerAbs = (flux * (1-(math.exp(-coefAl*metal_thick*nano_cm*densAl))))
    flux_to_diamond =flux-alLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-(math.exp(-coefC*diamon_thick*micro_cm*densC)))
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (alLayerAbs * 2 + cLayerAbs)*energy*enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    absAlall = (1-transAl) * 2
    absC = 1-transC
    return current , heatLoad, absAlall, absC

def MAdevice_perf_pt (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    coefPt = np.interp((energy*ToMev), pt[:,0], pt[:,2])
    transPt = (math.exp(-coefPt*metal_thick*nano_cm*densPt))
    ptLayerAbs = (flux * (1-transPt))
    flux_to_diamond =flux-ptLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-transC)
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    absPtall = (1-transPt) * 2
    absC = 1-transC
    return current , heatLoad, absPtall, absC

def Tran_device_perf_pt (energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy), carbonT[:,0], carbonT[:,1])
    coefPt = np.interp((energy), ptT[:,0], ptT[:,1])
    ptLayerAbs = (flux * (1-coefPt))
    flux_to_diamond =flux-ptLayerAbs
    cLayerAbs = flux_to_diamond * (1-coefC)
    current = (cLayerAbs * energy * enConv) /(freeCarrier)
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    absPtall = (1-coefPt)*2
    absC = 1-coefC
    return current , heatLoad, absPtall, absC

def Tran_device_perf_al (energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy), carbonT[:,0], carbonT[:,1])
    coefAl = np.interp((energy), alT[:,0], alT[:,1])
    alLayerAbs = (flux * (1-coefAl))
    flux_to_diamond =flux-alLayerAbs
    cLayerAbs = flux_to_diamond * (1-coefC)
    current = (cLayerAbs * energy * enConv)/(freeCarrier)
    absHeat = (alLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    return current , heatLoad

In [3]:
# old cell with stuff from other analysis 
MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# MAnumlow = MAdevice_perf_al(40,100,8000,1e15)
tranHigh = Tran_device_perf_pt (8000,5e8)
transAll = MAnumHigh[2] + MAnumHigh[3]
print(transAll)
# tranLow = Tran_device_perf_al (8000,1e15)
# print ("\nMass atenuation method high energy current is {:e} A, heat {:e} W.\n low energy current {:e} A, heat {:e} W\n".format(MAnumHigh[0],MAnumHigh[1],MAnumlow[0],MAnumlow[1]))
# print ("Transmission method high energy current is {:e} A, heat {:e} W.\n low energy current {:e} A, heat {:e} W\n".format(tranHigh[0], tranHigh[1], tranLow[0], tranLow[1]))
print("{:1.3e}".format(MAnumHigh[1]))
print("{:1.3e}".format(tranHigh[1]))

0.03963673595198114
4.719e-08
5.830e-08


In [4]:
# heat load calc for APS for Matt H 8/11/2023 

# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,20,23700,5.E15)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)



3.36e-03
1.56e-01
0.006472041749346125


In [5]:
# heat load calc for Denis Kane APS 
# energy range 3 to 35 keV, flux estimate 1e14 to 2e15 photons/s in a 0.5x0.5 mm area 

# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,20,50000,2E15)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)



4.12e-04
1.76e-02
0.0008428961251785161


In [6]:
# heat load calc for Gosia at CLS
# energy range 5 to 21 keV, flux estimate 5E12 photons/s in a 0.02x0.08 mm area 


# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

# MAnumHigh = MAdevice_perf_pt(45,20,10000,5E12)
MAnumHigh = MAdevice_perf_al(20,20,2000,1E12)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)



2.04e-05
4.81e-04
0.8741054579839661


In [7]:
# 278 W and the maximum power density will be 29 W/mrad2 (500mA ring current, B=0.99 T).
# 5 - 25 keV 

In [8]:
# heat load calc for Alexander at ???
# -	fixed energy of 60 keV
# -	bandwidth dE/E = 10^-4
# -	photon flux 10^10 ph/sec
# -	beamsize 100 µm - 1 mm
# -	position (center of mass) accuracy < 10 µm
# -	integrated intensity resolution < 1 %
# -	mild vacuum (10^-1 mbar, scroll pump)

# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,20,60000,1E10)
# MAnumHigh = MAdevice_perf_al(20,20,60000,1E10)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)



2.17e-09
7.98e-08
0.0006050556099993498


In [14]:
# heat load calc for Nicola at Elettra 
# -	5-20 keV
# -	bandwidth dE/E = 10^-4
# -	photon flux 10^12 - 10^13 ph/sec
# -	beamsize 5x5um2 up to 50x50um2
# pink beam 


# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,20,1500,1E13)
# MAnumHigh = MAdevice_perf_al(20,20,60000,1E10)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)



1.67e-04
4.27e-03
1.1619834047096869


In [10]:
# heat load calc for Pohang Accelerator white beam 
# -	5-25 keV
# -	photon flux 10^13 - 4*10^15 ph/sec
# -	beamsize 5.721 mm x 2.620 mm 
# white beam 


# (diamon_thick, metal_thick, energy, flux)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,30,5000,4.14E13)
# MAnumHigh = MAdevice_perf_al(20,20,60000,1E10)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)

5.61e-04
1.56e-02
0.3112238911388734


In [11]:
# heat load calc for Pohang Accelerator white beam 
# -	3.5-22 keV
# -	photon flux 10^14 - 4*10^14 ph/sec
# -	beamsize 500 x 50 um  


# (diamon_thick in microns, metal_thick in nm, energy eV, flux in ph/s)
# syntax example: > MAnumHigh = MAdevice_perf_pt(40,30,10000,5e8)
# returns: current, heatLoad, absPtall, absC 

MAnumHigh = MAdevice_perf_pt(45,30,3500,4E14)
# MAnumHigh = MAdevice_perf_al(20,20,60000,1E10)
current = MAnumHigh[0] * 1000
print("{:1.2e}".format(MAnumHigh[0])) ### current in Amperes
print("{:1.2e}".format(MAnumHigh[1])) ### heat load in W 
transAll = MAnumHigh[2] + MAnumHigh[3] ### not a percent, and this is actually absorption 
print(transAll)

9.14e-03
2.54e-01
0.7795713332638723
